In [1]:
import pickle
import numpy as np
import random
import string
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import get_file
from sklearn.model_selection import train_test_split
# from underthesea import word_tokenize
from nltk import word_tokenize
import nltk
nltk.download('punkt')
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = '/content/drive/MyDrive/NLP Training/'
# path = ''

In [4]:
real_estates_corpus = open(path + 'datasets/real_estates.txt', 'r', encoding='utf-8').read()
# big_corpus = open(path + 'datasets/big_corpus.txt', 'r', encoding='utf-8').read().lower() # just get 1mil char
# big_corpus = big_corpus[:len(big_corpus) // 100]
# corpus = real_estates_corpus + big_corpus
corpus = real_estates_corpus
print('Len of super big corpus: %d' % len(corpus))

Len of super big corpus: 551269


In [5]:
punct = string.punctuation
table = str.maketrans('', '', punct)
punct = punct.translate(table) # rermove punctuation
corpus = corpus.replace(r'\n', ' ')
# print(punct)
corpus = ''.join([i for i in corpus if i.isalnum() or i == ' ']) # just get the character
open(path + 'datasets/real_estates_preprocessed_2.txt', 'w', encoding='utf-8').write(corpus)

475235

In [6]:
# replace_with_2_space = open(path + 'datasets/replace_with_2_spaces.txt', 'r', encoding='utf-8').read().split('\n')
# replace_with_2_space = [i.split(' ', 1) for i in replace_with_2_space]
# for val in replace_with_2_space:
#   old = val[0]
#   new = val[1]
#   corpus = corpus.replace(old, new)

In [7]:
# tokens = word_tokenize(corpus) # split out
tokens = word_tokenize(corpus)
corpus = ' '.join(tokens) # insert 

In [8]:
char_set = list(set(corpus))
idx_char = {i: c for i, c in enumerate(char_set)}
char_idx = {c: i for i, c in enumerate(char_set)}

In [9]:
max_len = 50
step = 5
corpus_set = [corpus[i:i + max_len] for i in range(0, len(corpus) - max_len, step)]
next_char = [corpus[i + max_len] for i in range(0, len(corpus) - max_len, step)]

In [10]:
X = np.zeros((len(corpus_set), max_len, len(char_set)), dtype=np.bool)
y = np.zeros((len(next_char), len(char_set)), dtype=np.bool)
# X = np.array([[[1 if char_idx[char] == i else 0 for i in range(len(char_set))] for char in sentence] for sentence in corpus_set], dtype=np.bool)
# y = np.array([[1 if char_idx[char] == i else 0 for i in range(char_set)] for char in next_char], dtype=np.bool)
for i, sentence in enumerate(corpus_set):
    for j, char in enumerate(sentence):
        X[i, j, char_idx[char]] = True
    y[i, char_idx[next_char[i]]] = True

In [11]:
len(char_idx)

102

In [12]:
char_idx

{' ': 93,
 '0': 62,
 '1': 1,
 '2': 94,
 '3': 57,
 '4': 22,
 '5': 18,
 '6': 52,
 '7': 55,
 '8': 80,
 '9': 36,
 'a': 34,
 'b': 43,
 'c': 69,
 'd': 53,
 'e': 35,
 'f': 71,
 'g': 30,
 'h': 23,
 'i': 61,
 'j': 29,
 'k': 49,
 'l': 33,
 'm': 97,
 'n': 11,
 'o': 70,
 'p': 31,
 'q': 84,
 'r': 74,
 's': 2,
 't': 10,
 'u': 44,
 'v': 26,
 'w': 92,
 'x': 40,
 'y': 63,
 'z': 38,
 'à': 79,
 'á': 67,
 'â': 21,
 'ã': 6,
 'è': 89,
 'é': 99,
 'ê': 85,
 'ì': 9,
 'í': 96,
 'ò': 95,
 'ó': 54,
 'ô': 32,
 'õ': 16,
 'ù': 45,
 'ú': 68,
 'ý': 50,
 'ă': 51,
 'đ': 60,
 'ĩ': 42,
 'ũ': 14,
 'ơ': 100,
 'ư': 28,
 'ạ': 47,
 'ả': 46,
 'ấ': 20,
 'ầ': 86,
 'ẩ': 24,
 'ẫ': 82,
 'ậ': 0,
 'ắ': 56,
 'ằ': 58,
 'ẵ': 48,
 'ặ': 81,
 'ẹ': 8,
 'ẻ': 37,
 'ế': 13,
 'ề': 78,
 'ể': 25,
 'ễ': 73,
 'ệ': 15,
 'ỉ': 91,
 'ị': 3,
 'ọ': 64,
 'ỏ': 101,
 'ố': 98,
 'ồ': 19,
 'ổ': 7,
 'ỗ': 88,
 'ộ': 75,
 'ớ': 17,
 'ờ': 65,
 'ở': 27,
 'ỡ': 76,
 'ợ': 87,
 'ụ': 39,
 'ủ': 83,
 'ứ': 41,
 'ừ': 66,
 'ử': 5,
 'ữ': 90,
 'ự': 12,
 'ỳ': 4,
 'ỵ': 59,
 'ỷ': 72

In [13]:
X.shape, y.shape

((94184, 50, 102), (94184, 102))

In [21]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.01)
Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape

((93242, 50, 102), (942, 50, 102), (93242, 102), (942, 102))

In [22]:
batch_size = 64

In [23]:
print('Buid model...')
model = Sequential()
model.add(LSTM(256, input_shape=(max_len, len(char_set)), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(len(char_set), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Buid model...


In [24]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 50, 256)           367616    
_________________________________________________________________
dropout_2 (Dropout)          (None, 50, 256)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 102)               26214     
Total params: 919,142
Trainable params: 919,142
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.fit(Xtrain, ytrain, batch_size=batch_size, epochs=200)

Epoch 1/200
1457/1457 [==============================] - 17s 12ms/step - loss: 2.1274 - accuracy: 0.4369
Epoch 2/200
1457/1457 [==============================] - 17s 12ms/step - loss: 1.7135 - accuracy: 0.5433
Epoch 3/200
1457/1457 [==============================] - 17s 11ms/step - loss: 1.4542 - accuracy: 0.6103
Epoch 4/200
1457/1457 [==============================] - 17s 11ms/step - loss: 1.2648 - accuracy: 0.6620
Epoch 5/200
1457/1457 [==============================] - 17s 12ms/step - loss: 1.1220 - accuracy: 0.6993
Epoch 6/200
1457/1457 [==============================] - 17s 12ms/step - loss: 1.0015 - accuracy: 0.7320
Epoch 7/200
1457/1457 [==============================] - 17s 12ms/step - loss: 0.9071 - accuracy: 0.7551
Epoch 8/200
1457/1457 [==============================] - 17s 11ms/step - loss: 0.8265 - accuracy: 0.7762
Epoch 9/200
1457/1457 [==============================] - 17s 11ms/step - loss: 0.7591 - accuracy: 0.7926
Epoch 10/200
1457/1457 [==============================]

In [27]:
loss, acc = model.evaluate(Xtest, ytest, batch_size=batch_size)
print('Loss: %f' % loss)
print('Accuracy %f' % acc)

15/15 [==============================] - 1s 9ms/step - loss: 1.5488 - accuracy: 0.8132
Loss: 1.548829
Accuracy 0.813163


In [28]:
def sample(preds, temperature=0.5):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [29]:
model.save(path + 'models/lstm_model.h5')

In [30]:
import pickle

In [31]:
with open(path + 'pickles/idx_char.pkl', 'wb') as f:
  pickle.dump(idx_char, f)
  f.close()
with open(path + 'pickles/char_idx.pkl', 'wb') as f:
  pickle.dump(char_idx, f)
  f.close()

In [32]:
def create_value(seed_text, max_char=400):
  generated = seed_text
  for _ in range(max_char): # create max_char character
    value = np.zeros((1, max_len, len(char_set)))
    sentence = generated[-max_len:] # lấy max_len kí tự cuối 
    for i, char in enumerate(sentence):
      value[0, i, char_idx[char]] = True
    predict_value = model.predict(value, verbose=0)[0]
    predict_value = sample(predict_value)
    # predict_value = np.argmax(predict_value)
    nxt_char = idx_char[predict_value]
    generated += nxt_char
  return generated

In [33]:
txt = open(path + 'datasets/input.txt', 'r').read()
create_value(txt, 200)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


'mua nhà giá rẻpđđđ3đ8đđb33stđsđ544dqpiiddp5dddfddcmhanhadatquangminhcomvn còn nhiều nhà bán mới đẹp khác khu vực quận gò vấp phường 15 tân bình cho quý khách hàng thăm quan và chọn lựa nhadatquangminh nhabangovap n'